# CSV 기반 RAG (Retrieval-Augmented Generation) 시스템

## 핵심 개요
이 시스템은 CSV 파일에 저장된 고객 데이터를 기반으로 RAG (Retrieval-Augmented Generation) 시스템을 구축합니다.  
추천시스템에 사용될 가능성 높음 

## 주요 기능
- CSV 파일 로드 및 청킹 (Chunking)  
- FAISS + OpenAI 임베딩을 통한 벡터 스토어 생성  
- 검색 엔진 (Retriever) 설정  
- CSV 데이터에 대한 질의응답 (Q&A) 시스템 구축  


## 방법론
1.  CSV 전처리
- LangChain의 `CsvLoader`를 사용해 CSV 파일을 로드  
- 데이터를 특정 크기의 청크(Chunk)로 나눠 처리  


2.  벡터 스토어 생성
- OpenAI 임베딩을 이용해 각 청크를 벡터화  
- FAISS를 통해 고속 유사도 검색 최적화  

3.  Retriever 설정
- 사용자의 질의에 대해 가장 관련성 높은 청크를 반환하도록 설정  

 장점
- 대규모 데이터 처리: 청크 단위로 대규모 CSV 데이터 처리 가능  
- 빠른 검색 성능: FAISS를 통한 고속 유사도 검색  
- 유연한 확장성: 청크 크기 및 검색 결과 수 조정 가능  
- 최신 NLP 기술 활용: OpenAI 임베딩을 통해 강력한 의미 검색 지원  

결론

이 RAG 시스템은 CSV 기반 Q&A 시스템 구축에 강력한 기반을 제공
특히 고객 정보와 같이 구조화된 데이터에서 빠르고 정확한 정보 검색이 가능


In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [3]:
import pandas as pd

file_path = ('../data/customers-100.csv') 
data = pd.read_csv(file_path)
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


In [5]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [6]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# 임베딩 및 인덱스 생성 후 벡터 저장소 생성
embeddings = OpenAIEmbeddings()
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))
vector_store = FAISS(
    embedding_function=OpenAIEmbeddings(),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [7]:
vector_store.add_documents(documents=docs)

['60520d77-5bf0-405a-a6ab-3c6af65fd80e',
 'c235334c-c16c-413b-8e75-55ea4ebba631',
 'f171cd00-6f7f-4cfc-9ffb-8c9446ef0152',
 '2d6eb187-1e18-4ba8-b1cb-54730199650b',
 '1a4a2d90-c29d-4832-b0d7-2d7138b9f853',
 'd9fae9a2-9c12-4fd1-bb0c-e1fc87ac1f74',
 '4802c1e6-1086-467f-bf6c-d90ba0dead55',
 '94c7775d-67a8-41b0-b0d9-278a779fb27b',
 '0809b461-7283-4c1f-a62d-23ebf1af8769',
 'e919487d-544d-4d9e-94bc-6420366a6b17',
 'f150a346-ce47-4441-995f-9c137a097df5',
 'c0038b8e-046a-4248-8bad-5c46b71f20be',
 'e612f021-22cf-4fe3-a5dc-a8aeaf5a34b0',
 '8dd45080-106c-460a-b76c-aa992d2aa5fd',
 'ac81e82b-5565-460c-83c7-355e56ac9837',
 'c32da65c-9f86-4577-99b5-98ea12f9fe3a',
 'a1baf975-3ba3-47ce-838a-cca935f79749',
 '78731483-f9b8-4e32-94cf-7a6555478a4c',
 '9cab8e11-86ae-4e6c-809f-fcbe1a229af0',
 '7fcaf5c8-9170-446c-ab6b-68ddd093e384',
 '28ae0e7a-d49c-4756-82b9-aad30d9b8bf3',
 'bf79e14a-6455-4167-b7eb-39c5fdfd4a39',
 '76a3710a-bff0-47f9-82f1-6d931058001d',
 'a1dda2f9-e646-467e-9167-303d69676d96',
 '5c1cf0e6-b56f-

Create the retrieval chain

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [9]:
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

'Sheryl Baxter works for the Rasmussen Group.'